In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import netCDF4 as nc
from MRD import *
import time as clk
import xarray as xr
from cftime import num2pydate
import copy#pour copier des objets 

np.set_printoptions(threshold=10, edgeitems=2)
xr.set_options(display_expand_data=False)

%config InlineBackend.figure_format='retina'

repertoire_courant=os.getcwd()
print(repertoire_courant)

#on prend la tour 2, bas, ouest
fichier_T2_TO5=repertoire_courant+r'\T2_TO5.nc'
fichier_T2_right_east=repertoire_courant+r'\tower2-right-east.nc'
fichier_T2_snowfox=repertoire_courant+r'T2_snowfox_lf.dat'
t1=clk.time()
t0=clk.time()
print("chargement des données (cela peut prendre du temps)")

Ds_T2_TO5=xr.open_dataset(fichier_T2_TO5)
Ds_T2_TO5=Ds_T2_TO5.assign_coords({'index':Ds_T2_TO5["TIMESTAMP"].values}).rename({'index':'temps'}).drop_vars('TIMESTAMP')

f_ech=20.0#Hz

#delimitation de la periode d'etude
tmin=np.datetime64('2023-09-01 00:00:00')
tmax=np.datetime64('2023-09-08 13:00:00')
deg_to_K=lambda x:x.assign(TK=x['T']+273.15).drop_vars(['T']).rename({'TK':'T'})#convertit la température sonique en K

TO5_IRGA_1=deg_to_K(xr.merge((Ds_T2_TO5['Ux_1'],
                              Ds_T2_TO5['Uy_1'],
                              Ds_T2_TO5['Uz_1'],
                              Ds_T2_TO5['Ts_1'],
                              Ds_T2_TO5['H2O'],
                              Ds_T2_TO5['cell_tmpr'],
                              Ds_T2_TO5['cell_press'])).rename({'Ux_1':'u',
                                                                'Uy_1':'v',
                                                                'Uz_1':'w',
                                                                'Ts_1':'T'})).sel(temps=slice(tmin,tmax))

TO5_CSAT_2=deg_to_K(xr.merge((Ds_T2_TO5['Ux_2'],Ds_T2_TO5['Uy_2'],Ds_T2_TO5['Uz_2'],
                              Ds_T2_TO5['Ts_2'])).rename({'Ux_2':'u','Uy_2':'v','Uz_2':'w','Ts_2':'T'}).sel(temps=slice(tmin,tmax)))

TO5_CSAT_3=deg_to_K(xr.merge((Ds_T2_TO5['Ux_3'],Ds_T2_TO5['Uy_3'],Ds_T2_TO5['Uz_3'],
                      Ds_T2_TO5['Ts_3'])).rename({'Ux_3':'u','Uy_3':'v','Uz_3':'w','Ts_3':'T'}).sel(temps=slice(tmin,tmax)))

def len_2D(l):
    "retourne la longueur totale d'une liste 2D (exemple l=[[1,2],[3,3,5]])"
    cpt=0
    for i in range(0,len(l)):
        cpt+=len(l[i])
    return cpt

def slice_segments(array,taille_segments,equals=True):
    N_a=len(a)
    N_seg=N_a//taille_segments
    reste =N_a% taille_segments
    cpt=0
    res=[]
    while cpt <=N_seg:
        res.append(a[taille_segments*cpt:taille_segments*(cpt+1)])
        cpt+=1
    if (reste!=0) and (equals==True): res[-1]=np.append(res[-1],[np.nan for i in range(len(res[-1]),taille_segments)])
    return res
def Ri_B(U,V,T_z,T_s,z):
    "retourne le nombre de Richardson Bulk, avec T_z et T_s les températures à la hauteur z et en surface (en K)"
    return (9.81*(T_z-T_s)*z)/(T_s*(U**2+V**2))
def annulation_w(level):
    "applique une double rotation afin d'orienter le vecteur vitesse dans le sens du vent, de façon à avoir <w>=0"
    #Pour cela, on oriente  d'abord le repère dans le sens du vent, on annule <w>, puis on réoriente le repère dans le systeme de 
    #coordonnées horizontales du début
    new_level=copy.deepcopy(level)
    theta=np.degrees(np.arctan(np.mean(new_level['v'],axis=1)/np.mean(new_level['u'],axis=1)))
    new_level['u'],new_level['v']=rotation_segments(new_level['u'],new_level['v'],-theta)
    
    alpha=np.degrees(np.arctan(np.mean(new_level['w'],axis=1)/np.mean(new_level['u'],axis=1)))
    new_level['u'],new_level['w']=rotation_segments(new_level['u'],new_level['w'],-alpha)
    #theta=np.degrees(np.arctan(np.mean(new_level['v'],axis=1)/np.mean(new_level['u'],axis=1)))
    new_level['u'],new_level['v']=rotation_segments(new_level['u'],new_level['v'],theta)
    return new_level

print(f"fait en {clk.time()-t1}")
#print(Ds_T2_LW)
#print(Ds_T2_TO5)
#print(TO5_CSAT_3)
duree_segments='30min'
taille_segments=int(duree_segments.strip('min'))*60*20
print(TO5_IRGA_1)
print(TO5_CSAT_2)

C:\Users\evanl\Documents\Stage IGE
chargement des données (cela peut prendre du temps)
fait en 2.841954231262207
<xarray.Dataset>
Dimensions:     (temps: 13031678)
Coordinates:
  * temps       (temps) datetime64[ns] 2023-09-01 ... 2023-09-08T13:00:00
Data variables:
    u           (temps) float32 ...
    v           (temps) float32 ...
    w           (temps) float32 ...
    H2O         (temps) float32 ...
    cell_tmpr   (temps) float32 ...
    cell_press  (temps) float32 ...
    T           (temps) float32 278.7 278.5 278.4 278.1 ... 281.7 281.7 283.3
<xarray.Dataset>
Dimensions:  (temps: 13031678)
Coordinates:
  * temps    (temps) datetime64[ns] 2023-09-01 ... 2023-09-08T13:00:00
Data variables:
    u        (temps) float32 ...
    v        (temps) float32 ...
    w        (temps) float32 ...
    T        (temps) float32 278.0 277.7 277.9 277.9 ... 283.0 283.5 283.9 283.4


In [42]:
print(f"taille_segment {taille_segments}")
new_Ds=TO5_IRGA_1.coarsen(temps=taille_segments,boundary='trim').construct(temps=('temps', 'index'))
new_Ds_moy=new_Ds.mean('index')
print(new_Ds.isel(temps=0).values)
print(new_Ds_moy.isel(temps=0).values)
new_Ds_f=new_Ds-new_Ds_moy
print(new_Ds_f.isel(temps=0).values)

taille_segment 36000
<bound method Mapping.values of <xarray.Dataset>
Dimensions:     (index: 36000)
Coordinates:
    temps       (index) datetime64[ns] 2023-09-01 ... 2023-09-01T00:29:59.950000
Dimensions without coordinates: index
Data variables:
    u           (index) float32 -2.59 -2.489 -2.348 ... -1.922 -2.096 -2.089
    v           (index) float32 -2.585 -2.52 -2.487 ... 0.1877 -0.36 0.141
    w           (index) float32 0.3307 0.516 0.5281 ... 0.07148 0.6983 0.1062
    H2O         (index) float32 -0.845 -0.85 -0.8464 ... 0.8717 0.864 0.8593
    cell_tmpr   (index) float32 3.066 3.066 3.069 3.069 ... 3.926 3.926 3.926
    cell_press  (index) float32 74.05 74.06 74.06 74.06 ... 74.07 74.07 74.07
    T           (index) float32 278.7 278.5 278.4 278.1 ... 278.1 278.1 278.2>
<bound method Mapping.values of <xarray.Dataset>
Dimensions:     ()
Data variables:
    u           float32 -2.409
    v           float32 -2.152
    w           float32 0.3788
    H2O         float32 0.009069

In [37]:
def slice_segments_Ds(Ds,segment_size,dim='temps',new_dim_name='index',boundary='trim'):
    return Ds.coarsen({dim:segment_size},boundary=boundary).construct({dim:(dim,new_dim_name)})

<xarray.Dataset>
Dimensions:     (temps: 361, index: 36000)
Coordinates:
    temps       (temps, index) datetime64[ns] 2023-09-01 ... 2023-09-08T12:30...
Dimensions without coordinates: index
Data variables:
    u           (temps, index) float32 -2.59 -2.489 -2.348 ... -0.1034 -0.4336
    v           (temps, index) float32 -2.585 -2.52 -2.487 ... 4.659 4.862 4.628
    w           (temps, index) float32 0.3307 0.516 0.5281 ... -0.8372 -0.5952
    H2O         (temps, index) float32 -0.845 -0.85 -0.8464 ... 3.222 3.242
    cell_tmpr   (temps, index) float32 3.066 3.066 3.069 ... 10.09 10.09 10.09
    cell_press  (temps, index) float32 74.05 74.06 74.06 ... 74.65 74.65 74.65
    T           (temps, index) float32 278.7 278.5 278.4 ... 282.2 282.0 281.9
